In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

<font style = 'font-family:Book Antiqua'><font size = 3>
    <i>Unit 2.2 Project 7</i>
    
<font style = 'font-family:Book Antiqua'><font size = 7>
    <b>Challenge: Feedback Analysis</b>

<font style = 'font-family:Book Antiqua'><font size = 5><b>Assignment:</b>
    
<font size = 4>Pick one of the company data files and build your own classifier. When you're satisfied with its performance (at this point just using the accuracy measure shown in the example), test it on one of the other datasets to see how well these kinds of classifiers translate from one context to another.

Include your model and a brief writeup of your feature engineering and selection process to submit and review with your mentor.
<hr width=50%>

In [2]:
# Importing packages
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

<font style = 'font-family:Book Antiquea'><font size = 5><b>Challenge:</b>
    
<font size = 4> For this challenge, I have chosen to use the Amazon data set from <a href='https://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences'>UCI learning dataset of sentiment labelled sentences</a>.    

In [3]:
# Grabbing and process the raw data.
amazon = ('amazon_cells_labelled.txt')
amazon = pd.read_csv(amazon, delimiter= '\t', header=None)
amazon.columns = ['sentence', 'score'] # 1 = positive 0 = negative

amazon.head()

,sentence,score
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


<font style = 'font-family:Book Antiquea'><font size = 4>In the dataset, we can see that there are two columns, a sentence and a label. From the guided example in the previous section, we know that a sentence is not really a feature and is not super helpful. In this case, the two columns would be used to predict whether a review is positive or negative. 

<font style = 'font-family:Book Antiquea'><font size = 4><b>Learning the Data Set</b>

<font style = 'font-family:Book Antiquea'><font size = 4>Below I am analyzing the dataset to see what descriptive words or phrases I can use to determine if a review is positive.

In [4]:
amazon['score'] = (amazon['score']==1)

In [5]:
# Searching for patterns in the dataset and standardizing the data to one format.
amazon['sentence'] = amazon.sentence.str.replace(r'[^a-zA-Z\d\s:]', '')
amazon['sentence'] = amazon['sentence'].str.lower()

In [6]:
# split negative messages and combine into one list
positive_words = amazon.sentence[amazon.score].str.cat(sep=' ').split()

negative_words = amazon.sentence[amazon.score == False].str.cat(sep=' ').split()

<font style = 'font-family:Book Antiqua'><font size = 4>Below is the count of unique positive and negative words that one can use to analyze if a review is good or bad.

In [7]:
print ('Number of unique negative words:',len(np.unique(negative_words)))

Number of unique negative words: 1293


In [8]:
print ('Number of unique postive words:',len(np.unique(positive_words)))

Number of unique postive words: 1137


<font style ='font-family:Book Antiqua'><font size =4>In the two code blocks below, I am creating two arrays. The first array contains all the postive keywords in the column <b><i>sentence</i></b>. The second array returns the unique values that are not in either array.

In [9]:
# Creating a data frame to house positive keywords.
print('Positve keywords array:')
keywords = pd.Series(positive_words).value_counts().keys().ravel()

keywords

Positve keywords array:


array(['the', 'and', 'i', ..., 'division', 'install', 'lasts'],
      dtype=object)

In [10]:
print('Unique values array:')
diff = np.setdiff1d(
        ar1=pd.Series(positive_words).value_counts().keys().ravel(), 
                      ar2=pd.Series(negative_words).value_counts().keys().ravel()
                     )
diff

Unique values array:


array(['100', '15', '18', '2000', '2005', '24', '2mp', '325', '350', '42',
       '5020', '680', '7', '700w', '8530', ':', ':oh', 'ac', 'accessable',
       'accessing', 'accompanied', 'according', 'activesync', 'adapters',
       'address', 'adorable', 'against', 'alarm', 'allot', 'allow',
       'allowing', 'alot', 'aluminum', 'amazed', 'amazing', 'ample',
       'ant', 'antiglare', 'anywhere', 'apart', 'appears', 'applifies',
       'appointments', 'armband', 'arrival', 'ask', 'aspect', 'attacked',
       'attractive', 'authentic', 'autoanswer', 'available', 'awesome',
       'awsome', 'basic', 'batteries', 'beat', 'beats', 'beautiful',
       'behing', 'bitpim', 'blackberry', 'blacktop', 'blueant',
       'bluetoothmotorola', 'bluetooths', 'boot', 'bose', 'boy',
       'brilliant', 'browser', 'browsing', 'bt250v', 'bubbling', 'build',
       'bulky', 'cables', 'called', 'canal', 'cancellation', 'cancelling',
       'capability', 'capacity', 'carried', 'carries', 'carry', 'cases',
 

In [11]:
# Creating a new feature using keywords from above array.
new = amazon
keywords = diff

for key in keywords:
    # Note that we add spaces around the key so that we're getting the word,
    # not just pattern matching.
    new[str(key)] = new.sentence.str.contains(
        ' ' + str(key) + ' ',
        case=False
    )

In [12]:
# Specifying an outcome for model below.
data = new[keywords]
target = new['score']

<font style = 'font-family:Book Antiqua'><font size = 4>Lastly, we run a test on the model to determine the accuracy.

In [13]:
from sklearn.naive_bayes import BernoulliNB

bnb = BernoulliNB()

bnb.fit(data, target)

y_pred = bnb.predict(data)

print("Number of mislabeled points out of a total {} points : {}\nAccuracy: {}%".format(
    data.shape[0],
    (target != y_pred).sum(), round((1 - (target != y_pred).sum()/data.shape[0]) * 100, 2)
))

Number of mislabeled points out of a total 1000 points : 302
Accuracy: 69.8%


<hr width=50%>

<font style = 'font-family:Book Antiquea'><font size = 5><b>Run the same model on another dataset (Yelp)</b>

<font style = 'font-family:Book Antiqua'><font size = 4>For the second part of the assignment, I will run the same test on another data set (YelP) to determine how well the classifers in the Amazon dataset translate.

In [14]:
yelp = pd.read_csv('yelp_labelled.txt', delimiter= '\t', header=None)
yelp.columns = ['review', 'score']
yelp.head()

,review,score
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [15]:
yelp['score'] = (yelp['score'] == 1)
yelp['review'] = yelp.review.str.replace(r'[^a-zA-Z\d\s:]', '')
yelp['review'] = yelp['review'].str.lower()

# split negative messages and combine into one list
apositive_words = yelp.review[yelp.score].str.cat(sep=' ').split()

anegative_words = yelp.review[yelp.score == False].str.cat(sep=' ').split()

In [16]:
print ('Number of unique negative words:',len(np.unique(anegative_words)))
print ('Number of unique negative words:',len(np.unique(apositive_words)))

Number of unique negative words: 1397
Number of unique negative words: 1246


In [17]:
akeywords = pd.Series(anegative_words).value_counts().keys().ravel()

adiff = np.setdiff1d(
        ar1=pd.Series(apositive_words).value_counts().keys().ravel(), 
                      ar2=pd.Series(anegative_words).value_counts().keys().ravel()
                     )
adiff

array(['2007', '23', '6', '7', '70', 'absolute', 'absolutley', 'accident',
       'accommodations', 'accomodate', 'accordingly', 'across', 'added',
       'affordable', 'afternoon', 'airport', 'almonds', 'amazingrge',
       'ambience', 'ample', 'andddd', 'app', 'appetizers', 'approval',
       'aria', 'array', 'assure', 'atmosphere1', 'auju', 'awesome',
       'ayce', 'az', 'baba', 'bacon', 'baklava', 'bank', 'bargain',
       'bartender', 'bartenders', 'baseball', 'bbq', 'bean', 'beateous',
       'beautiful', 'beautifully', 'beauty', 'beers', 'bellies', 'belly',
       'biscuit', 'bits', 'blanket', 'block', 'bloddy', 'blue', 'bone',
       'booksomethats', 'bowl', 'boxes', 'boyfriend', 'boys', 'bread',
       'breakfastlunch', 'breeze', 'brick', 'brings', 'bruschetta',
       'buffets', 'buldogis', 'bunch', 'butter', 'caballeros', 'caesar',
       'caf', 'cakeohhh', 'calligraphy', 'cannoli', 'cape', 'capers',
       'caring', 'carpaccio', 'case', 'cavier', 'chai', 'charming',
      

In [18]:
new = yelp
akeywords = adiff

for key in akeywords:
    # Note that we add spaces around the key so that we're getting the word,
    # not just pattern matching.
    new[str(key)] = new.review.str.contains(
        ' ' + str(key) + ' ',
        case=False
    )

In [19]:
data = new[akeywords]
target = new['score']

In [20]:
from sklearn.naive_bayes import BernoulliNB

bnb = BernoulliNB()

bnb.fit(data, target)

y_pred = bnb.predict(data)

print("Number of mislabeled points out of a total {} points : {}\nAccuracy: {}%".format(
    data.shape[0],
    (target != y_pred).sum(), round((1 - (target != y_pred).sum()/data.shape[0]) * 100, 2)
))

Number of mislabeled points out of a total 1000 points : 294
Accuracy: 70.6%
